In [1]:
import pandas as pd
from tqdm import tqdm
#from google.colab import drive
import os
from matplotlib import style
style.use('fivethirtyeight')
from fuzzywuzzy import fuzz, process
import numpy as np

C:\Anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

In [3]:
lq =  pd.read_csv('bcstores.csv')
print('The shape: %d x %d' % lq.shape)
print('There are %d unique values' % lq.name.shape[0])

The shape: 501 x 8
There are 501 unique values


In [4]:
beer = pd.read_csv('beeradvocate.csv')

In [5]:
beer.comb = beer.mnf.str.lower()+ ' '+ beer.name.str.lower()
beer.comb = beer.comb.str.lower().str.replace("brewing","").str.replace("brewery","")

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [8]:
beer.shape

(513, 11)

# Drop duplicates: diff packing, diff type( can or bottle)

In [9]:
lq.name = lq.name.str.lower().str.replace(" tall can","").str.replace(" can","").str.replace(" 4-pack","").str.replace(" 6-pack","")

In [10]:
lq = lq.sort_values(['name','price']).drop_duplicates(subset = ['name'], keep = 'first')

In [11]:
lq.shape

(413, 8)

# Drop variety pack

In [12]:
lq.drop(labels = lq[lq['style'].str.contains('Variety')].index, inplace = True)

In [13]:
# is this necessary?
def split_str(string):
    return set(string.split(" "))

In [13]:
lq.desc = lq.desc.apply(lambda x: x.replace("\\n","").replace("\\xa0",""))

In [ ]:
lq[lq.name.str.contains('parallel 49')]

In [19]:
lq.iloc[[246]]

,name,country,price,sku_link,alc,style,beer_type,desc
327,parallel 49 - tricycle grapefruit radler,Canada,$11.79,/product/526897,3.5%,Radler,Domestic Beer - BC Craft Beer,Grapefruit beer made with Basic hops and malts used for brewing a lager and fortified with natural grapefruit juice.\n


In [17]:
# recheck?
lq.drop(index = [8,173,179,193,194,199,213,235,246,261,333,426,366], inplace = True)

In [18]:
lq.shape

(378, 8)

In [19]:
#drop rows with duplicate description
# em sua cai 'No info' lai cho dung roi
temp1 = lq[lq.desc != 'No info'].sort_values(['desc','price'], ascending = False).drop_duplicates(subset = ['desc'], keep = 'first')

In [20]:
lq_cleaned = pd.concat([lq[lq.desc == 'No info'],temp1])

In [22]:
lq_cleaned.sort_index(inplace = True)

In [21]:
lq_cleaned.shape

(372, 8)

# Extract lq1

In [29]:
#ua tai sao chi filter cai national domestic beer ra ko vay chi
lq1 = lq_cleaned[lq_cleaned.beer_type !='Domestic Beer - National Domestic Beer']

In [30]:
lq1.shape

(293, 8)

In [32]:
def ratio_per_word (word, string):
    ratio = process.extract(word, string, limit = 1, scorer = fuzz.token_set_ratio)

    #sort dictionary by values
    simi = ratio[0][1] 
    print('simi: ',simi)
    rate = ratio[0][0]
    print('rate: ',rate)
    return rate, simi

In [33]:
long_rate=[]
long_simi = []
for i in lq1.name:
    rate,simi = ratio_per_word(i,beer.comb)
    long_rate.append(rate)
    long_simi.append(simi)

simi:  91
rate:  33 acres  co. 33 acres of life
simi:  93
rate:  33 acres  co. 33 acres of nirvana
simi:  92
rate:  33 acres  co. 33 acres of ocean
simi:  93
rate:  33 acres  co. 33 acres of sunshine
simi:  100
rate:  33 acres  co. 33 acres of sunshine
simi:  100
rate:  grupo cervezas alhambra, sl reserva 1925
simi:  88
rate:  asahi breweries ltd asahi super dry
simi:  94
rate:  balderdash  co. sneaky weasel
simi:  100
rate:  baltika breweries baltika #7 export
simi:  65
rate:  bard's tale beer company bard's gold
simi:  100
rate:  barkerville  co. 52 foot stout
simi:  70
rate:  bomber  pale ale
simi:  50
rate:   lobkowicz lobkowicz baron
simi:  60
rate:  swinkels family brewers bavaria beer
simi:  86
rate:  beau's all natural  company beau's - full time ipa tall cans
simi:  43
rate:  brauerei beck & co. beck's
simi:  86
rate:  big rock urban  and eatery traditional ale
simi:  100
rate:  bitburger brauerei premium pils (premium beer)
simi:  100
rate:  bomber  park life passion fruit al

simi:  88
rate:  off the rail  company classic pale ale
simi:  48
rate:  bard's tale beer company bard's gold
simi:  95
rate:  old yale  co. moon dance mango wheat
simi:  100
rate:  oskar blues grill & brew dale's pale ale
simi:  86
rate:  pacific western  company canterbury dark mild
simi:  100
rate:  pacific western  company cariboo blonde
simi:  100
rate:  pacific western  company cariboo genuine draft
simi:  100
rate:  pacific western  company cariboo honey lager
simi:  100
rate:  pacific western  company cariboo malt
simi:  70
rate:  pacific western  company dude
simi:  100
rate:  pacific western  company pacific pilsner
simi:  100
rate:  grupo modelo s.a. de c.v. cerveza pacifico clara
simi:  100
rate:  paderborner isenbeck brauerei haus cramer gmbh & co. kg paderborner pilsener
simi:  90
rate:  parallel 49  company apricotopus
simi:  100
rate:  parallel 49  company craft lager
simi:  79
rate:  parallel 49  company craft lager
simi:  77
rate:  parallel 49  company craft lager
sim

In [34]:
len(long_rate)

293

In [35]:
len(long_simi)

293

In [36]:
lq1['beercomb'] = long_rate
lq1['simi'] = long_simi
lq1['confirm']= np.nan

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [37]:
for i in lq1['beercomb']:
    print(len(i))

30
33
31
34
34
40
35
29
35
36
30
16
26
36
60
26
42
46
35
21
41
41
32
29
44
40
11
25
26
36
36
21
32
23
64
39
38
38
38
38
14
55
55
45
21
27
16
38
36
29
31
17
69
22
31
16
25
40
51
51
31
21
48
35
26
40
39
40
24
24
40
30
34
34
25
48
45
49
52
47
62
64
39
30
53
24
37
43
54
39
37
33
32
32
50
31
38
44
16
22
32
14
25
31
35
23
35
26
35
38
29
27
41
45
45
49
45
35
64
38
22
57
33
39
34
41
33
27
43
51
52
37
44
24
28
41
24
25
23
39
31
31
31
38
45
45
15
45
19
38
53
43
20
38
36
36
40
45
39
46
44
37
29
40
48
76
32
32
32
32
37
31
34
61
32
47
18
17
18
56
65
65
30
20
29
27
47
42
44
29
32
27
30
15
32
33
15
40
33
48
19
64
64
64
64
83
50
33
33
33
36
38
37
33
26
26
32
39
79
27
26
35
56
33
42
42
43
43
43
31
43
27
27
31
43
31
37
31
58
22
61
23
25
25
26
49
37
41
50
40
43
32
30
24
21
30
33
57
57
46
39
21
49
26
42
29
32
37
24
29
24
24
48
44
43
37
48
16
37
45
38
34
56
16
34
27
56
45
51
34
25
27
41


In [72]:
# what does this part do?
lq1['beercomb'] = lq1['beercomb'].apply(lambda x: x[0] if len(x)==1 else x)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [38]:
lq1.loc[(lq1.simi>=80),'confirm'] = 'yes'

C:\Anaconda\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [39]:
lq1.loc[(lq1.simi<70),'confirm'] = 'no'

In [40]:
lq1.reset_index(inplace = True)

In [41]:
lq1

,index,name,country,price,sku_link,alc,style,beer_type,desc,beercomb,simi,confirm
0,0,33 acres of life lager,Canada,$12.29,/product/772335,4.8%,California Common Or Steam,Domestic Beer - BC Craft Beer,"1848, West Coast convictions of resourcefulness and persistence, the California Common was born. Easy on the alcohol and weighing in on the taste for a complete balance. Fruit like quality is created by fermenting the lager at ale temperatures. Hints at a full-bodied ale yet retains a crisp finish. \n",33 acres co. 33 acres of life,91,yes
1,1,33 acres of nirvana 6 pack,Canada,$13.49,/product/880930,7%,India Pale Ale,Domestic Beer - BC Craft Beer,33 Acres is one of Vancouver’s most well-loved craft beer brewers. This West Coast Indian Pale Ale beautifully balances bitter-fresh pine notes with grapefruit rind and candied orange citrus. Try it with some fried chicken for a true taste of nirvana.\n,33 acres co. 33 acres of nirvana,93,yes
2,2,33 acres of ocean pale ale,Canada,$12.29,/product/837591,5.6%,Pale Ale,Domestic Beer - BC Craft Beer,"A full flavoured beer integrated with a distinct floral hop gives a quality unique to our Pacific Northwest surroundings. This style is evolutionary from a IPA with an assertive hop, lower alcohol content, and slight pine aroma. Low in malt, this beer has a refined quality and a refreshing temperament.\n",33 acres co. 33 acres of ocean,92,yes
3,3,33 acres of sunshine blanche,Canada,$12.29,/product/90837,5%,Wheat,Domestic Beer - BC Craft Beer,"Thanks to summers' launching acclaims, this romantic French Blanchè can willingly co-star any seasons current scenes. Showcasing flare for the prismatic, 33 Acres of Sunshine synthesizes an unfiltered wheat dream. Joining the arrays, glowing hops of Aurora and Styrian Golding awaken subtle anise seeds, dazzling coriander, and candied orange ovations. \n",33 acres co. 33 acres of sunshine,93,yes
4,4,33 acres of sunshine,Canada,$21.99,/product/23859,5%,Wheat,Domestic Beer - BC Craft Beer,No info,33 acres co. 33 acres of sunshine,100,yes
5,7,alhambra reserva 1925,Spain,$13.29,/product/638759,6.4%,Pilsner,Import Beer,This award-winning lager is brewed over 35 days. The nose is full-on caramel leading to a full-flavoured palate and a satisfying finish.\n,"grupo cervezas alhambra, sl reserva 1925",100,yes
6,9,asahi super dry big,Japan,$14.79,/product/541201,5%,Pilsner,Import Beer,No info,asahi breweries ltd asahi super dry,88,yes
7,13,balderdash brewing - sneaky weasel craft lager,Canada,$1.99,/product/198948,5.6%,Lager,Domestic Beer - BC Craft Beer,"Now in a tall can! This mildly hopped craft lager delivers a bold 5.6% alc/vol, but goes down crisp and clean. Pairs well with game day celebrations, fun in the sun and good times with friends.\n",balderdash co. sneaky weasel,94,yes
8,14,baltika 7,Russia,$2.49,/product/176909,5.4%,Lager,Import Beer,No info,baltika breweries baltika #7 export,100,yes
9,15,bards tale - gold gluten free lager,USA,$15.29,/product/193590,4.7%,Gluten-free,Import Beer,"100% gluten-free lager brewed with malted sorghum provides a traditional beer flavour. It has a medium clear, amber/gold colour, moderate head, aromas of roasted malt and toasted grains with sweet fruit notes. It's dry and light-bodied with balanced hops and flavours of apple, caramel and sweet grain.\n",bard's tale beer company bard's gold,65,no


In [84]:
lq1.shape

(85, 12)

In [77]:
lq1[(lq1.simi>=70) & (lq1.simi<80)]

,index,name,country,price,sku_link,alc,style,beer_type,desc,beercomb,simi,confirm
0,25,belgian moon - white ale 4,Canada,$11.49,/product/183512,5.45%,Wheat,Domestic Beer - National Domestic Beer,No info,lighthouse company belgian white,70,NaN
3,56,coors organic,Canada,$12.79,/product/57910,3.8%,Lager,Domestic Beer - National Domestic Beer,No info,dogwood organic ipa,70,NaN
25,383,shock top belgian white,Canada,$25.99,/product/51868,5.2%,Wheat,Domestic Beer - National Domestic Beer,"Unfiltered Belgian-style white ale that is naturally cloudy with a billowy foam head and light golden hue. Slight taste of citrus peel and coriander, with a smooth, refreshing finish.\n",lighthouse company belgian white,72,NaN
26,400,stanley park brewing - concession stand,Canada,$22.49,/product/119357,5%,Other,Domestic Beer - National Domestic Beer,"Try our new rotational mix pack featuring our SunSetter Peach Wheat Ale and the new Brewpub Lager! 3 x SunSetter Peach Wheat Ale, 3 x Trail Hopper IPA, 3 x Daytrip West Coast Lager, 3 x Brewpub Lager",stanley park layer up,73,NaN
27,407,stanley park brewing - windstorm,Canada,$2.39,/product/198832,5.4%,Pale Ale,Domestic Beer - National Domestic Beer,"Tropical fruit and citrus hop character. Medium body, full flavour. And .10 cents from every Tall Can sold benefits the Stanley Park Ecology Society!\n",stanley park layer up,73,NaN
28,399,stanley park brewing - 1897 amber,Canada,$11.99,/product/255573,5.1%,Amber Ale,Domestic Beer - National Domestic Beer,This traditional pub-style amber ale has soft caramel aromas and toasted malt flavour. \n,stanley park layer up,73,NaN
31,406,stanley park brewing - waypoint hazy pale ale,Canada,$11.99,/product/102158,5.2%,Pale Ale,Domestic Beer - National Domestic Beer,This Hazy Pale Ale has grapefruit citrus and tropical fruit flavours and aromas. It is medium bodied with a low perceived bitterness.,stanley park west coast pale ale,79,NaN
38,119,granville island - sip easy,Canada,$10.99,/product/92773,4.3%,Lager,Domestic Beer - National Domestic Beer,"Supremely sessionable and refreshing, Sip Easy Lager is the ideal balance between full flavour and easy drinking. It's no compromises, no concessions � just 120 calories per serving, it's crafted for those who just want to keep it easy.\n",granville island west coast ipa,79,NaN
39,268,molson - old style pilsner,Canada,$10.49,/product/904540,5%,Pilsner,Domestic Beer - National Domestic Beer,"Staying true to it's Western Canada roots, Old Style Pilsner is a well-balanced, clean, crisp lager beer with medium body, smooth palate and distinctive hoppy aroma and finish.\n","lakefront , inc. new grist gluten-free pilsner style beer",70,NaN
40,401,stanley park brewing - daytrip,Canada,$11.99,/product/4278,4.4%,Lager,Domestic Beer - National Domestic Beer,Smooth tasting and sessionable. Light citrus hop character. \n,stanley park layer up,73,NaN


In [81]:
#wrong code
lq1.iloc[[2,50,51,72,77,94,170,171,175,177,178,179],11] = 'yes'

IndexError: positional indexers are out-of-bounds

In [ ]:
lq1.loc[lq1.confirm.isna(),'confirm']='no'

In [42]:
lq1.iloc[:,[4,9,10,11]]

,sku_link,beercomb,simi,confirm
0,/product/772335,33 acres co. 33 acres of life,91,yes
1,/product/880930,33 acres co. 33 acres of nirvana,93,yes
2,/product/837591,33 acres co. 33 acres of ocean,92,yes
3,/product/90837,33 acres co. 33 acres of sunshine,93,yes
4,/product/23859,33 acres co. 33 acres of sunshine,100,yes
5,/product/638759,"grupo cervezas alhambra, sl reserva 1925",100,yes
6,/product/541201,asahi breweries ltd asahi super dry,88,yes
7,/product/198948,balderdash co. sneaky weasel,94,yes
8,/product/176909,baltika breweries baltika #7 export,100,yes
9,/product/193590,bard's tale beer company bard's gold,65,no


In [43]:
df = lq.merge(lq1.iloc[:,[4,9,10,11]], on = 'sku_link', how = 'left')

In [45]:
lq_final = df

In [46]:
lq_final

,name,country,price,sku_link,alc,style,beer_type,desc,beercomb,simi,confirm
0,33 acres of life lager,Canada,$12.29,/product/772335,4.8%,California Common Or Steam,Domestic Beer - BC Craft Beer,"1848, West Coast convictions of resourcefulness and persistence, the California Common was born. Easy on the alcohol and weighing in on the taste for a complete balance. Fruit like quality is created by fermenting the lager at ale temperatures. Hints at a full-bodied ale yet retains a crisp finish. \n",33 acres co. 33 acres of life,91.0,yes
1,33 acres of nirvana 6 pack,Canada,$13.49,/product/880930,7%,India Pale Ale,Domestic Beer - BC Craft Beer,33 Acres is one of Vancouver’s most well-loved craft beer brewers. This West Coast Indian Pale Ale beautifully balances bitter-fresh pine notes with grapefruit rind and candied orange citrus. Try it with some fried chicken for a true taste of nirvana.\n,33 acres co. 33 acres of nirvana,93.0,yes
2,33 acres of ocean pale ale,Canada,$12.29,/product/837591,5.6%,Pale Ale,Domestic Beer - BC Craft Beer,"A full flavoured beer integrated with a distinct floral hop gives a quality unique to our Pacific Northwest surroundings. This style is evolutionary from a IPA with an assertive hop, lower alcohol content, and slight pine aroma. Low in malt, this beer has a refined quality and a refreshing temperament.\n",33 acres co. 33 acres of ocean,92.0,yes
3,33 acres of sunshine,Canada,$21.99,/product/23859,5%,Wheat,Domestic Beer - BC Craft Beer,No info,33 acres co. 33 acres of sunshine,100.0,yes
4,33 acres of sunshine blanche,Canada,$12.29,/product/90837,5%,Wheat,Domestic Beer - BC Craft Beer,"Thanks to summers' launching acclaims, this romantic French Blanchè can willingly co-star any seasons current scenes. Showcasing flare for the prismatic, 33 Acres of Sunshine synthesizes an unfiltered wheat dream. Joining the arrays, glowing hops of Aurora and Styrian Golding awaken subtle anise seeds, dazzling coriander, and candied orange ovations. \n",33 acres co. 33 acres of sunshine,93.0,yes
5,alexander keith's - india pale ale,Canada,$14.99,/product/687251,5%,India Pale Ale,Domestic Beer - National Domestic Beer,"Brewed in Halifax since 1820, India Pale Ale is light in colour and hopped in flavour. Only the lightest and finest barleys that produce a pale malt are used while the amounts of hops are increased so as to give a pronounced hop flavour.\n",NaN,NaN,NaN
6,alhambra reserva 1925,Spain,$13.29,/product/638759,6.4%,Pilsner,Import Beer,This award-winning lager is brewed over 35 days. The nose is full-on caramel leading to a full-flavoured palate and a satisfying finish.\n,"grupo cervezas alhambra, sl reserva 1925",100.0,yes
7,asahi super dry big,Japan,$14.79,/product/541201,5%,Pilsner,Import Beer,No info,asahi breweries ltd asahi super dry,88.0,yes
8,balderdash brewing - sneaky weasel craft lager,Canada,$1.99,/product/198948,5.6%,Lager,Domestic Beer - BC Craft Beer,"Now in a tall can! This mildly hopped craft lager delivers a bold 5.6% alc/vol, but goes down crisp and clean. Pairs well with game day celebrations, fun in the sun and good times with friends.\n",balderdash co. sneaky weasel,94.0,yes
9,baltika 7,Russia,$2.49,/product/176909,5.4%,Lager,Import Beer,No info,baltika breweries baltika #7 export,100.0,yes


In [ ]:
lq.loc[lq.confirm.isna(),'confirm']='no'

In [ ]:
lq1['beercomb2'] = long_rate

In [ ]:
lq1

In [ ]:
lq[lq.confirm =='no']